## Imports

In [1]:
import cupy as cp
import numpy as np
from pathlib import Path
from sparse_recon.sparse_deconv import sparse_deconv
from cucim.skimage.exposure import match_histograms
import napari
import zarr
import gc

## Load data

In [2]:
raw_data_path = Path(r'Z:\20211217-franky\MT-suc70-2nM-04\timelapse_2021_12_17-08_58_36\deskew_output\OPM_processed.zarr')
raw_data = zarr.open(str(raw_data_path),'r',dtype=np.uint16)
t_slices,ch_slices,z_slices,y_slices,x_slices = raw_data.shape

## Perform sparse deconvolution

In [12]:
gc.collect()
cp.clear_memo()

t_slices=10

sparse_data_t01 = np.zeros((t_slices,z_slices,y_slices,x_slices),dtype=np.float32)
bkd = 0

for t_idx in np.arange(0,t_slices):

    print("time %d/%d" % (t_idx, t_slices))

    cp_raw_data_t0 = cp.asarray(raw_data[0,0,:] - bkd)
    cp_raw_data_t0[cp_raw_data_t0<0]=0
    cp_raw_data_t0_scale = cp_raw_data_t0.max()
    cp_raw_data_t0 = cp_raw_data_t0/cp_raw_data_t0_scale

    cp_raw_data_t_idx = cp.asarray(raw_data[t_idx,0,:] - bkd)
    cp_raw_data_t_idx[cp_raw_data_t_idx<0]=0
    cp_raw_data_t_idx_scale = cp_raw_data_t_idx.max()
    cp_raw_data_t_idx = cp_raw_data_t_idx/cp_raw_data_t_idx_scale
    cp_raw_data_t_idx = match_histograms(cp_raw_data_t_idx,cp_raw_data_t0)
    raw_data_eq = cp.asnumpy(cp_raw_data_t_idx_scale * cp_raw_data_t_idx)

    del cp_raw_data_t0, cp_raw_data_t0_scale, cp_raw_data_t_idx, cp_raw_data_t_idx_scale
    gc.collect()
    cp.clear_memo()

    sparse_data_t01[t_idx,:] = sparse_deconv(raw_data_eq, [.220/ .115, .220/.115],prior=-1,sparse_iter=100,tcontinuity=3.0,fidelity=100,sparsity=5,deconv_iter=10,deconv_type=3,up_sample=0)
    
    del raw_data_eq

    gc.collect()
    cp.clear_memo()


time 0/10
Start the Sparse deconvolution...
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
sparse hessian time 362.89s
deconv time 5.05s
time 1/10
Start the Sparse deconvolution...
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
sparse hessian time 360.70s
deconv time 5.11s
time 2/10
Start the Sparse deconvolution...
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
sparse hessian time 361.62s
deconv time 4.97s
time 3/10
Start the Sparse deconvolution...
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
Start the Sparse deconvolution...
99 iterations done
sparse hessian time 361.61s
deconv time 4.89s
time

## View raw OPM and sparse deconvolved OPM in Napari

In [13]:
viewer = napari.Viewer()

#viewer.add_image(raw_data[0:10,0,:],scale=[.115, 0.115,.115],colormap='bop purple',name="raw data")
#viewer.add_image(sparse_data,scale=[.115, 0.115,.115],colormap='bop blue',blending='additive')
viewer.add_image(sparse_data_t01,scale=[.115, 0.115,.115],colormap='bop orange',name="sparse decon",blending='additive')

viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'